In [1]:
import nltk
nltk.download('stopwords')
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\Admin\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\Admin\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\Admin\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [5]:
train_df=pd.read_csv(r'C:\Users\Admin\Desktop\train.csv')
test_df=pd.read_csv(r'C:\Users\Admin\Desktop\test.csv')


In [6]:
train_df.head(2)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1


In [7]:
train_df.isnull().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [8]:
train_df['keyword']=train_df['keyword'].fillna('None')
train_df['location']=train_df['location'].fillna('Unknown')

In [9]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
# Function to clean text
def clean_text(text):
    text= text.lower()  # Convert to lowercase
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters and numbers
    return text

In [11]:
# Clean the text
train_df['cleaned_text'] = train_df['text'].apply(clean_text)


In [12]:
import nltk

# ... your existing code ...

# Download 'punkt_tab' before using word_tokenize
nltk.download('punkt_tab')

# Function to remove stopwords
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    return ' '.join([word for word in words if word not in stop_words])

# ... your existing code ...

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [22]:
# Function for stemming
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
nltk.download('punkt')
stemmer = LancasterStemmer()
def stem_text(text):
   words = word_tokenize(text)
   return ' '.join([stemmer.stem(word) for word in words])

train_df['cleaned_text'] = train_df['cleaned_text'].apply(stem_text)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
# Function for lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
   words = word_tokenize(text)
   return ' '.join([lemmatizer.lemmatize(word) for word in words])

# Apply lemmatization
train_df['cleaned_text'] = train_df['cleaned_text'].apply(lemmatize_text)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
# Tokenize the cleaned text for Word2Vec
tokenized_text = [word_tokenize(text) for text in train_df['cleaned_text']]

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4)

# Save the model for future use
word2vec_model.save("word2vec.model")

In [29]:
# Function to get average Word2Vec vectors for each text
def get_vector(text):
    words = word_tokenize(text)
    vector = np.zeros(100)  # Assuming vector_size=100
    count = 0
    for word in words:
        if word in word2vec_model.wv:
            vector += word2vec_model.wv[word]
            count += 1
    if count > 0:
        vector /= count
    return vector

# Apply the function to get vectors
train_df['text_vector'] = train_df['cleaned_text'].apply(get_vector)

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Prepare features and labels
X = np.array(train_df['text_vector'].tolist())
y = train_df['target']

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.7202889034799738


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression



# Train a model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.6940249507550886


In [32]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import MinMaxScaler  # Import MinMaxScaler for scaling
from sklearn.pipeline import Pipeline # Import the Pipeline class

# ... your existing code ...

# Train a model with scaling
# Create a pipeline to first scale the data and then apply MultinomialNB
model = Pipeline([('scaler', MinMaxScaler()), ('clf', MultinomialNB())])
model.fit(X_train, y_train)  # Train the model

# Evaluate the model
accuracy = model.score(X_test, y_test)
print(f'Accuracy: {accuracy}')

Accuracy: 0.582403151674327
